In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # ==============================
# # Konfigurasi Path dan Proporsi BARU
# # ==============================
# # Path ke dataset asli (folder yang berisi 30 kelas)
# BASE_DIR = '/content/drive/MyDrive/Deep-Learning/Food-Image-Classification/Food-image-classification/Fruit-And-Vegetable-Diseases-Dataset'

# # Path untuk direktori baru (Hanya 2 kelas: Healthy, Rotten)
# # Dibuat di luar Split_Dataset lama
# NEW_BASE_DIR = '/content/drive/MyDrive/Deep-Learning/Food-Image-Classification/Binary_Split_Dataset'

# # Proporsi pembagian (tetap sama)
# TRAIN_SPLIT = 0.8
# VAL_SPLIT = 0.1
# TEST_SPLIT = 0.1

# # ==============================
# # Fungsi Utama Binary Splitting
# # ==============================
# def split_data_binary(base_source_dir, base_target_dir, split_ratio):
#     """
#     Menggabungkan semua gambar dari 30 sub-folder menjadi 2 kelas (Healthy/Rotten)
#     dan membaginya menjadi Train, Val, Test.
#     """

#     # Inisialisasi folder 2 kelas di Train/Val/Test
#     for set_name in ['train', 'validation', 'test']:
#         os.makedirs(os.path.join(base_target_dir, set_name, 'Healthy'), exist_ok=True)
#         os.makedirs(os.path.join(base_target_dir, set_name, 'Rotten'), exist_ok=True)

#     # Dapatkan daftar semua folder kelas lama (misal: Apple_Healthy, Tomato_Rotten)
#     old_class_names = [d for d in os.listdir(base_source_dir)
#                        if os.path.isdir(os.path.join(base_source_dir, d))]

#     if not old_class_names:
#         print("\nERROR: Tidak ada sub-direktori kelas yang ditemukan di BASE_DIR.")
#         return

#     # Dictionary untuk mengumpulkan semua file berdasarkan kondisi (Healthy/Rotten)
#     all_files_by_condition = {'Healthy': [], 'Rotten': []}

#     for class_name in old_class_names:
#         source_path = os.path.join(base_source_dir, class_name)

#         # Tentukan kondisi baru: Healthy atau Rotten
#         if class_name.endswith('_Healthy'):
#             condition = 'Healthy'
#         elif class_name.endswith('_Rotten'):
#             condition = 'Rotten'
#         else:
#             print(f"Mengabaikan folder: {class_name}")
#             continue

#         # Kumpulkan semua file di folder ini
#         current_files = [(filename, class_name) for filename in os.listdir(source_path)
#                          if os.path.isfile(os.path.join(source_path, filename))]

#         all_files_by_condition[condition].extend(current_files)
#         print(f"Mengumpulkan {len(current_files)} gambar dari {class_name} ke {condition}.")


#     # ==================================
#     # 2. Pembagian dan Penyalinan Data
#     # ==================================
#     for condition, file_list in all_files_by_condition.items():
#         random.shuffle(file_list) # Acak semua file

#         # Hitung pembagian
#         total_count = len(file_list)
#         train_count = int(total_count * split_ratio['train'])
#         val_count = int(total_count * split_ratio['val'])

#         train_files = file_list[:train_count]
#         val_files = file_list[train_count:train_count + val_count]
#         test_files = file_list[train_count + val_count:]

#         print(f"\nKondisi: {condition} (Total: {total_count} gambar)")
#         print(f"  Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")

#         # Salin file
#         for files, set_name in zip([train_files, val_files, test_files], ['train', 'validation', 'test']):
#             target_dir = os.path.join(base_target_dir, set_name, condition)
#             for filename, old_folder in files:
#                 source_file = os.path.join(base_source_dir, old_folder, filename)
#                 target_file = os.path.join(target_dir, f"{old_folder}_{filename}") # Beri nama unik
#                 shutil.copy(source_file, target_file)


# # ==============================
# # Jalankan Skrip Utama
# # ==============================
# if __name__ == '__main__':
#     # --- LOGIKA PENGHAPUSAN OTOMATIS ---
#     if os.path.exists(NEW_BASE_DIR):
#         print(f"Direktori lama '{NEW_BASE_DIR}' ditemukan. Menghapus direktori lama...")
#         shutil.rmtree(NEW_BASE_DIR)
#         print("Direktori lama berhasil dihapus.")

#     print(f"\nMembuat direktori baru untuk Binary Classification: {NEW_BASE_DIR}")
#     os.makedirs(NEW_BASE_DIR)

#     split_data_binary(BASE_DIR, NEW_BASE_DIR,
#                       split_ratio={'train': TRAIN_SPLIT, 'val': VAL_SPLIT, 'test': TEST_SPLIT})

#     print("\n=======================================================")
#     print("Proses pembagian dataset BINARY (Healthy/Rotten) selesai!")
#     print(f"Dataset baru Anda berada di: {NEW_BASE_DIR}")
#     print("=======================================================")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32
# NUM_CLASSES = 4 (Diabaikan di sini, tetapi di model akan menjadi 1)

# Path baru setelah binary split
train_dir_binary = '/content/drive/MyDrive/Deep-Learning/Food-Image-Classification/Binary_Split_Dataset/train'
validation_dir_binary = '/content/drive/MyDrive/Deep-Learning/Food-Image-Classification/Binary_Split_Dataset/validation'
test_dir_binary = '/content/drive/MyDrive/Deep-Learning/Food-Image-Classification/Binary_Split_Dataset/test'


# =======================================================
# 1. Image Data Generator untuk Training (Augmentasi Aggressive)
# =======================================================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2],
)

train_generator = train_datagen.flow_from_directory(
    train_dir_binary, # PATH BARU
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # KUNCI: Diubah menjadi 'binary'
    shuffle=True
)

# =======================================================
# 2. Image Data Generator untuk Validation dan Test (Hanya Normalisasi)
# =======================================================
validation_test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_test_datagen.flow_from_directory(
    validation_dir_binary, # PATH BARU
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # KUNCI: Diubah menjadi 'binary'
    shuffle=False
)

test_generator = validation_test_datagen.flow_from_directory(
    test_dir_binary, # PATH BARU
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # KUNCI: Diubah menjadi 'binary'
    shuffle=False
)

Found 23421 images belonging to 2 classes.
Found 2926 images belonging to 2 classes.
Found 2930 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# --- 1. DEFINISI PARAMETER DAN MODEL (Sederhana) ---
IMAGE_SIZE = (150, 150)
NUM_CLASSES = 1 # Wajib untuk biner

model = Sequential()

# Lapisan 1: Blok Fitur Dasar (Filters dikurangi sedikit)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))

# Lapisan 2: Blok Fitur Menengah
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.2)) # Mengurangi Dropout

# Lapisan 3: Blok Fitur Kompleks (Cukup)
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
# Lapisan 4 Dihapus!

# Lapisan Fully Connected
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001))) # Mengurangi unit Dense dari 256 menjadi 128
model.add(BatchNormalization())
model.add(Dropout(0.5)) # Mengurangi Dropout dari 0.7 menjadi 0.5

# Lapisan Output (Wajib: 1 unit dan Sigmoid)
model.add(Dense(NUM_CLASSES, activation='sigmoid'))

# --- 2. KOMPILASI MODEL ---
# Learning Rate bisa sedikit dinaikkan karena tugasnya lebih mudah
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Ringkasan model yang disederhanakan
model.summary()

# --- 3. CALLBACKS (Dapat dipertahankan, karena sudah efektif) ---
checkpoint_filepath = '/content/drive/MyDrive/Deep-Learning/Food-Image-Classification/best_model_checkpoint/best_fruit_disease_cnn_binary_simple.keras'

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
    restore_best_weights=True
)

lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

callbacks_list = [model_checkpoint_callback, early_stopping_callback, lr_reducer]

# --- 4. PELATIHAN MODEL ---
# ASUMSI: train_generator dan validation_generator sudah disesuaikan
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALIDATION = validation_generator.n // validation_generator.batch_size
EPOCHS = 15

print("Memulai pelatihan model (Binary Classification: Sederhana)...")

history = model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VALIDATION,
    callbacks=callbacks_list
)

print("\nPelatihan selesai! Model terbaik telah disimpan.")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 148, 148, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 74, 74, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 74, 74, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 37, 37, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     5,308,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,403,329 (20.61 MB)

 Trainable params: 5,402,625 (20.61 MB)

...

Epoch 1/15
133/731 ━━━━━━━━━━━━━━━━━━━━ 2:17:56 14s/step - accuracy: 0.5818 - loss: 1.1885

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


731/731 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - accuracy: 0.6513 - loss: 0.9897 
Epoch 1: val_accuracy improved from -inf to 0.68613, saving model to /content/drive/MyDrive/Deep-Learning/Food-Image-Classification/best_model_checkpoint/best_fruit_disease_cnn_binary_simple.keras
731/731 ━━━━━━━━━━━━━━━━━━━━ 12075s 17s/step - accuracy: 0.6513 - loss: 0.9895 - val_accuracy: 0.6861 - val_loss: 0.7483 - learning_rate: 5.0000e-04
Epoch 2/15
  1/731 ━━━━━━━━━━━━━━━━━━━━ 51:51 4s/step - accuracy: 0.6875 - loss: 0.8815

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy did not improve from 0.68613
731/731 ━━━━━━━━━━━━━━━━━━━━ 86s 111ms/step - accuracy: 0.6875 - loss: 0.8815 - val_accuracy: 0.6703 - val_loss: 0.7864 - learning_rate: 5.0000e-04
Epoch 3/15
128/731 ━━━━━━━━━━━━━━━━━━━━ 28:30 3s/step - accuracy: 0.7617 - loss: 0.6918

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt

# Ambil data akurasi dan loss dari objek history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Tentukan jumlah epoch
epochs = range(len(acc))

# ------------------------------------
# Plot Akurasi Training dan Validation
# ------------------------------------
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'r', label='Akurasi Training')
plt.plot(epochs, val_acc, 'b', label='Akurasi Validation')
plt.title('Akurasi Training dan Validation')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend(loc=0)

# ------------------------------------
# Plot Loss Training dan Validation
# ------------------------------------
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'r', label='Loss Training')
plt.plot(epochs, val_loss, 'b', label='Loss Validation')
plt.title('Loss Training dan Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc=0)

plt.show()

In [ ]:
# Tentukan langkah untuk evaluasi (Total Test Images / Batch Size)
STEP_SIZE_TEST = test_generator.n // test_generator.batch_size

print("Mengevaluasi model pada data test...")

# Melakukan evaluasi model
loss_test, acc_test = model.evaluate(
    test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1
)

print("\n=======================================================")
print(f"Hasil Evaluasi Akhir pada Test Set (Kriteria 5):")
print(f"Loss Test: {loss_test:.4f}")
print(f"Akurasi Test: {acc_test*100:.2f}%")
print("=======================================================")